Whole notebook has been run on Google colab pro,

created by: Michał Chrzanowski 2022

important note, if model stucks around 25% accuracy restart whole model and calculate again

In [15]:
#modules not used in this notebook but may come in handy later

# image processing of images
from skimage.color import rgb2hsv, hsv2rgb
def color_isolate(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    img_hsv = rgb2hsv(img[:,:,:3])   
    #MASK
    mask_1 = img_hsv [:,:,0] > 200/360
    mask_2 = img_hsv [:,:,0] < 280/360
    saturation_1 = img_hsv [:,:,1] > 0.35
    saturation_2 = img_hsv [:,:,1] < 0.70
    
    mask= mask_1*mask_1*saturation_1*saturation_2

    image_filtered = np.dstack((img[:,:,0]*mask,
                                img[:,:,1]*mask,
                                img[:,:,2]*mask))

    return image_filtered

#used to preview images loaded in model
def imshow(img):
    npimg = img.numpy()[0]
    plt.imshow(np.transpose(npimg, (1, 2, 0)), interpolation='nearest')

In [16]:
# can be omitted if mean and std is already calculated
# for calcualting mean and std od dataset
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in dataloader:
        # Mean over batch, height and width, but not over the channels
        channels_sum += torch.mean(data, dim=[0,2,3])
        channels_squared_sum += torch.mean(data**2, dim=[0,2,3])
        num_batches += 1

    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

#print(get_mean_and_std(train_loader))
#print(get_mean_and_std(test_loader))

In [17]:
!lsb_release -a #Linux info
!cat /proc/cpuinfo # CPU info
!df -h #disc info
!cat /proc/meminfo #RAM info
!sudo apt install speedtest-cli #library for speedtesting
!speedtest-cli # network speed test

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 18.04.6 LTS
Release:	18.04
Codename:	bionic
processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0x1
cpu MHz		: 2000.126
cache size	: 39424 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat m

In [18]:
!nvidia-smi #GPU info

Fri Sep 30 11:34:27 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    28W /  70W |   5420MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
# I decided to use external plotting service Weights and biases. It provides easy live preview of model state and  allows convenient runs comparasion
!pip install wandb
!wandb login 33cccacf373072ab6a5edc82d4770dddf40d42fb
import wandb

#mount google drive
from google.colab import drive
drive.mount('/content/drive')
#copy data from Google drive to internal machine drive for faster loading
google_path = "/content/drive/MyDrive/CNN_Data"
train_path = "/content/data"
!cp -R "$google_path" $train_path

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Imports
import time
import torch
import torchvision  # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.transforms as transforms  # Transformations and augmentations
from torch import optim  # For optimizers
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader #Dataloader module
from torch.utils.data import Dataset # Dataset module
import torchvision.transforms as transforms
from tqdm import tqdm #nice progress bar
import pandas as pd
import numpy as np
from skimage.io import imshow, imread

# Libraries for dataloader
from os import listdir
from os.path import isfile, join
from PIL import Image
import cv2
#Checking whether GPU RAM is empty
print('allocated CUDA memory: ',torch.cuda.memory_allocated())
print('cached CUDA memory: ',torch.cuda.memory_cached())

allocated CUDA memory:  0
cached CUDA memory:  0


/usr/local/lib/python3.7/dist-packages/torch/cuda/memory.py:393: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  FutureWarning)


In [3]:
# DATALOADERS

# Train Loader
class BloodCell_train(Dataset):
    def __init__(self, transform):
        self.labels = []
        self.dirName_1 = '/content/data/Blood_Cell_Images/dataset2-master/dataset2-master/images/TRAIN/EOSINOPHIL' # path to training data with EOSINOPHILS cells
        self.dirName_2 = '/content/data/Blood_Cell_Images/dataset2-master/dataset2-master/images/TRAIN/LYMPHOCYTE' # path to training data with LYMPHOCYTES cells
        self.dirName_3 = '/content/data/Blood_Cell_Images/dataset2-master/dataset2-master/images/TRAIN/MONOCYTE' # path to training data with MONOCYTES cells
        self.dirName_4 = '/content/data/Blood_Cell_Images/dataset2-master/dataset2-master/images/TRAIN/NEUTROPHIL' # path to training data with NEUTROPHILS cells
        self.files_names = []
        self.transform = transform
        self.load_images()
    

    def load_images(self):

        # create list of paths to images
        files_1 = [join(self.dirName_1, f) for f in listdir(self.dirName_1) if isfile(join(self.dirName_1, f))]
        files_2 = [join(self.dirName_2, f) for f in listdir(self.dirName_2) if isfile(join(self.dirName_2, f))]
        files_3 = [join(self.dirName_3, f) for f in listdir(self.dirName_3) if isfile(join(self.dirName_3, f))]
        files_4 = [join(self.dirName_4, f) for f in listdir(self.dirName_4) if isfile(join(self.dirName_4, f))]
        

        # connect lists of different cells in one big list
        self.files_names = files_1 + files_2 + files_3 + files_4

        # creating label list
        for file_name in tqdm(files_1):
            self.labels.append(0)

        for file_name in tqdm(files_2):
            self.labels.append(1)

        for file_name in tqdm(files_3):
            self.labels.append(2)

        for file_name in tqdm(files_4):
            self.labels.append(3)

        del files_1
        del files_2
        del files_3
        del files_4

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        image=cv2.imread(self.files_names[index])
        #image = color_isolate(image.astype(np.float32))
        y_label = self.labels[index]

        if self.transform:
            image = self.transform(image)

        return (image, y_label)

# Test Loader
class BloodCell_test(Dataset):
    def __init__(self, transform):
        self.labels = []
        self.dirName_1 = '/content/data/Blood_Cell_Images/dataset2-master/dataset2-master/images/TEST/EOSINOPHIL' # path to test data with EOSINOPHILS cells
        self.dirName_2 = '/content/data/Blood_Cell_Images/dataset2-master/dataset2-master/images/TEST/LYMPHOCYTE' # path to test data with LYMPHOCYTES cells
        self.dirName_3 = '/content/data/Blood_Cell_Images/dataset2-master/dataset2-master/images/TEST/MONOCYTE' # path to test data with MONOCYTES cells
        self.dirName_4 = '/content/data/Blood_Cell_Images/dataset2-master/dataset2-master/images/TEST/NEUTROPHIL' # path to test data with NEUTROPHILS cells
        self.files_names = []
        self.transform = transform
        self.load_images()
        

    def load_images(self):

        # create list of paths to images
        files_1 = [join(self.dirName_1, f) for f in listdir(self.dirName_1) if isfile(join(self.dirName_1, f))]
        files_2 = [join(self.dirName_2, f) for f in listdir(self.dirName_2) if isfile(join(self.dirName_2, f))]
        files_3 = [join(self.dirName_3, f) for f in listdir(self.dirName_3) if isfile(join(self.dirName_3, f))]
        files_4 = [join(self.dirName_4, f) for f in listdir(self.dirName_4) if isfile(join(self.dirName_4, f))]
        
        # connect lists of different cells in one big list
        self.files_names = files_1 + files_2 + files_3 + files_4

        # creating label list
        for file_name in tqdm(files_1):
            self.labels.append(0)

        for file_name in tqdm(files_2):
            self.labels.append(1)

        for file_name in tqdm(files_3):
            self.labels.append(2)

        for file_name in tqdm(files_4):
            self.labels.append(3)

        del files_1
        del files_2
        del files_3
        del files_4
        
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        image=cv2.imread(self.files_names[index])
        #image = color_isolate(image.astype(np.float32))
        y_label = self.labels[index]

        if self.transform:
            image = self.transform(image)

        return (image, y_label)


In [4]:
# model creation
class CNN(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=9,
            kernel_size=(11, 11),
            stride=(1, 1),
            padding=(1,1),
        )
        self.conv2 = nn.Conv2d(
            in_channels=9,
            out_channels=27,
            kernel_size=(7, 7),
            stride=(1, 1),
            padding=(0,0),
        )
        self.conv3 = nn.Conv2d(
            in_channels=27,
            out_channels=54,
            kernel_size=(5, 5),
            stride=(1, 1),
            padding=(0,0),
        )
        self.conv4 = nn.Conv2d(
            in_channels=54,
            out_channels=54,
            kernel_size=(5, 5),
            stride=(1, 1),
            padding=(1,1),
        )
        self.conv5 = nn.Conv2d(
            in_channels=54,
            out_channels=108,
            kernel_size=(5, 5),
            stride=(1, 1),
            padding=(0,0),
        )
        self.conv6 = nn.Conv2d(
            in_channels=108,
            out_channels=108,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(0,0),
        )
        self.conv7 = nn.Conv2d(
            in_channels=108,
            out_channels=108,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(0,0),
        )
        self.conv8 = nn.Conv2d(
            in_channels=108,
            out_channels=108,
            kernel_size=(3, 3),
            stride=(1, 1),
            padding=(0,0),
        )

        self.pool5 = nn.MaxPool2d(kernel_size=(5, 5), stride=(5, 5))
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        self.drop1d = nn.Dropout(0.5)
        self.BatchNorm2d1 = nn.BatchNorm2d(num_features=9)
        self.BatchNorm2d3 = nn.BatchNorm2d(num_features=54)
        self.fc1 = nn.Linear(41472, 5000)
        self.fc2 = nn.Linear(5000, 5000)
        self.fc3 = nn.Linear(5000, 5000)
        self.fc4 = nn.Linear(5000, num_classes)

    def forward(self, x):
        x = F.relu(self.BatchNorm2d1(self.conv1(x)))
        x = F.relu(self.conv2(x))
        x = F.relu(self.BatchNorm2d3(self.conv3(x)))
        x = F.relu(self.conv4(x))
        x = self.pool5(x)
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = self.pool2(x)
        x = F.relu(self.conv8(x))

        x = x.reshape(x.shape[0], -1)

        x = self.fc1(x)
        #x = self.drop1d(x)
        x = self.fc2(x)
        #x = self.drop1d(x)
        x = self.fc3(x)
        #x = self.drop1d(x)
        x = self.fc4(x)
        return x

In [5]:
# calculate outputs shape of layers
W=320
H=240
print('initial shape',W,H)
#1st conv
W=(W-11+2*1)/1+1
H=(H-11+2*1)/1+1
print('after 1st conv',W,H)
# 2st conv
W=(W-7+2*0)/1+1
H=(H-7+2*0)/1+1
print('after 2nd conv',W,H)
# 3nd conv
W=(W-5+2*0)/1+1
H=(H-5+2*0)/1+1
print('after 3rd conv',W,H)
# 4nd conv#
W=(W-5+2*1)/1+1
H=(H-5+2*1)/1+1
print('after 4th conv',W,H)
# pool5
W=W/5
H=H/5
print('after 1st pooling',W,H)
# 5nd conv
W=(W-5+2*0)/1+1
H=(H-5+2*0)/1+1
print('after 5th conv',W,H)
# 6nd conv
W=(W-3+2*0)/1+1
H=(H-3+2*0)/1+1
print('after 6th conv',W,H)
# 7nd conv
W=(W-3+2*0)/1+1
H=(H-3+2*0)/1+1
print('after 7th conv',W,H)
# pool2
W=W/2
H=H/2
print('after 2nd pooling',W,H)
# 8nd conv
W=(W-3+2*0)/1+1
H=(H-3+2*0)/1+1
print('after 8th conv',W,H)

number_out_channels = 108
print('Size of end output',H*W*number_out_channels)


initial shape 320 240
after 1st conv 312.0 232.0
after 2nd conv 306.0 226.0
after 3rd conv 302.0 222.0
after 4th conv 300.0 220.0
after 1st pooling 60.0 44.0
after 5th conv 56.0 40.0
after 6th conv 54.0 38.0
after 7th conv 52.0 36.0
after 2nd pooling 26.0 18.0
after 8th conv 24.0 16.0
Size of end output 41472.0


In [6]:
torch.cuda.empty_cache()
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'using: {device}')
torch.backends.cudnn.benchmark = True #let cudnn chose most efficient way of calculating convulsions

# Hyper-parameters
in_channels = 3
num_classes = 4
learning_rate = 3e-4
batch_size = 32
num_epochs = 100
weight_decay = 0.001

#Connenting to wandb project to log data there
import wandb
wandb.init(project="CNN_BloodCell_1")

# Initialize network
model = CNN(in_channels=in_channels, num_classes=num_classes)
model.to(device)

#transform_train = transforms.ToTensor()
#transform_test = transforms.ToTensor() 

# define transformations for datasets, stds and means have been calculated before with cell above on dataloaders without normalization
transform_train = transforms.Compose([transforms.ToTensor(),
                                      transforms.Normalize([0.6605, 0.6413, 0.6786],
                                                           [0.2612, 0.2636, 0.2644])])
transform_test = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize([0.6610, 0.6406, 0.6795],
                                                          [0.2606, 0.2623, 0.2636])])

#initialize datasets and dataloaders
dataset_train = BloodCell_train(transform=transform_train)
dataset_test = BloodCell_test(transform=transform_test)

# set shuffle = True to randomize order
# To avoid blocking computation code with data loading we set num_workers = 2
# pin_memory = True will automatically put the fetched data Tensors in pinned memory, and thus enables faster data transfer to CUDA-enabled GPUs.
train_loader = DataLoader(dataset=dataset_train, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(dataset=dataset_test, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

using: cuda


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chrzanowski000. Use `wandb login --relogin` to force relogin


100%|██████████| 624/624 [00:00<00:00, 267174.94it/s]


In [7]:
mode_par_num = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Number of trained parameters in model: " + '{:,}'.format(mode_par_num).replace(',', ' '))

Number of trained parameters in model: 257 980 958


In [8]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in tqdm(loader):
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples

In [9]:
from datetime import datetime
def save_model(test_acc):
    dic_out = {'model_params': model.state_dict(), 'optimizer_params': optimizer.state_dict()}
    df = pd.DataFrame.from_dict(dic_out)

    # datetime object containing current date and time
    now = datetime.now()
    print("now =", now)
    dt_string = now.strftime("%d_%m_%Y__%H_%M_%S")

    # save model dict on Google drive with time signature
    path = f'/content/drive/MyDrive/ML/CNN_BloodCell_{dt_string}_{test_acc}.csv'
    df.to_csv(path, index = False, header=True)

In [10]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
start_time = time.time()
# Optional
wandb.watch(model, criterion, log="all", log_freq=400)
train_acc_best = 0
test_acc_best = 0
# Train Network
for epoch in range(num_epochs):
    print(f'epoch nr {epoch}')
    for data, targets in tqdm(train_loader):
        # Get data to device
        data = data.to(device=device)
        targets = targets.to(device=device)


        # setting parameteres gradients to None, takes less memory than setting to '0'
        for param in model.parameters():
            param.grad = None

        # forward and loss calculation
        scores=model(data)
        loss = criterion(scores, targets)
        # log loss to wandb
        wandb.log({"loss": loss})

        # backward
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
    
    # Calculate Accuracy and save model if test acc increased
    train_acc = check_accuracy(train_loader, model)*100
    test_acc = check_accuracy(test_loader, model)*100
    if test_acc > test_acc_best:
        save_model(test_acc)
        test_acc_best = test_acc
        train_acc_best = train_acc
        print(f'model saved: train_acc:{train_acc_best}, test_acc:{test_acc_best}')

    # log train_acc and test_acc to wandb
    wandb.log({"train_acc": train_acc})
    wandb.log({"test_acc": test_acc})
    print(f"Accuracy on training set: {train_acc:.2f}")    
    print(f"Accuracy on test set: {test_acc:.2f}")

    #print('cached CUDA memory: ',torch.cuda.memory_cached())
print("--- %s seconds ---" % (time.time() - start_time))
torch.cuda.empty_cache()
print(f"Highest accuracy on training set: {train_acc_best}")
print(f"Highest accuracy on test set: {test_acc_best}")

epoch nr 0


100%|██████████| 78/78 [00:16<00:00,  4.72it/s]


now = 2022-09-30 11:56:54.531343
model saved: train_acc:38.45535659790039, test_acc:42.581424713134766
Accuracy on training set: 38.46
Accuracy on test set: 42.58
epoch nr 1


100%|██████████| 78/78 [00:14<00:00,  5.28it/s]


now = 2022-09-30 12:01:12.258122
model saved: train_acc:51.411067962646484, test_acc:57.137115478515625
Accuracy on training set: 51.41
Accuracy on test set: 57.14
epoch nr 2


...

Accuracy on training set: 99.15
Accuracy on test set: 87.21
epoch nr 32


100%|██████████| 78/78 [00:14<00:00,  5.54it/s]


Accuracy on training set: 99.13
Accuracy on test set: 85.40
epoch nr 33


  3%|▎         | 9/312 [00:05<03:03,  1.65it/s]


KeyboardInterrupt: ignored